### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd#импорт библиотеки Pandas
scoring = pd.read_csv('/datasets/data.csv')#чтение файла
scoring.info()#получим общую информацию о данных в таблице
scoring.head(10)#получим первые 10 строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Всего в таблице 12 столбцов, есть следующие типы данных: int64, float64, object. Сразу виден один артефакт - это отрицательное количество дней трудового стажа, притом, что есть и положительные значения, которые намного больше всех остальных. Есть предположение, что трудовой стаж больших значений выгружен не в днях, а в часах. Возможно это техническая ошибка при выгрузке данных. Более понятно этот столбец выглядел бы, если перевести дни в года (часы в года для больших значений) и заменить отрицательные значения на положительные. Но для проекта анализ трудового стажа не нужен, поэтому менять его на основании предположений не будем. Количество значений в столбцах таблицы различается. Значит есть пропущенные, при этом количество пропущенных значений в столбцах "Общий трудовой стаж в днях" и "Ежемесячный доход" одинаковое. Каждая строка таблицы содержит данные о платежеспособности определенного клиента, но ID клиентов не указано, следовательно, вполне возможны дубликаты. По первым 10 строкам наблюдается разный регистр ввода данных, надо проверить каждый столбец. Для проверки рабочих гипотез особенно важными столбцами являются столбы "children", "debt", "family_status", "total_income", "purpose".

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
scoring.columns#проверим название столбцов таблицы

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [3]:
scoring.isnull().sum()#отсутствующие значения

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#поcмотрим основные статистические характеристики данных по каждому числовому признаку 
scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
scoring[scoring['total_income'].isnull()].head()#выведем первые 5 строк с отсутствующими значениями по доходу

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [6]:
#находим медиану по доходу
total_income_median = scoring['total_income'].median()
total_income_median

145017.93753253992

In [7]:
#заменим пропуски в доходе на значение медианы
scoring['total_income'] = scoring['total_income'].fillna(value = total_income_median)

In [8]:
#заменим пропущенные значения в общем трудовом стаже на значения "неизвестно"
scoring['days_employed'] = scoring['days_employed'].fillna('unknown')

In [9]:
#проверяем остались ли пропущенные значения
scoring[scoring['total_income'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [10]:
#найдем минимальное и максимальное значения по "детям":
print('Минимальное значение по "children"', scoring['children'].min(), '\nМаксимальное значение по "children"', scoring['children'].max())
#дополнительно посмотрим все уникальные значения
scoring['children'].unique()

Минимальное значение по "children" -1 
Максимальное значение по "children" 20


array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [11]:
#заменим отрицательное значение на положительное
scoring['children'] = scoring['children'].abs()

In [12]:
#проверим сколько строк со значением 20
scoring[scoring['children'] == 20]['children'].count()

76

In [13]:
#заменим все значения 20 на 2
scoring['children'] = scoring['children'].replace(20, 2)

In [14]:
#проверим замену на 2 и замену на положительные значения
scoring['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [15]:
scoring['dob_years'].unique()#проверим уникальные значения возраста клиентов

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [16]:
#находим среднее арифметическое по возрасту
dob_years_mean = scoring[scoring['dob_years'] != 0]['dob_years'].mean()
dob_years_mean
#заменим нулевые значения на среднее арифметическое
scoring['dob_years'] = scoring['dob_years'].replace(0, dob_years_mean)
scoring['dob_years'].describe()#проверим исправления 

count    21525.000000
mean        43.497479
std         12.218166
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [17]:
scoring['education'] = scoring['education'].str.lower()#приводим столбец к нижнему регистру
scoring['education'].unique()#проверяем изменения

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [18]:
scoring['family_status'].unique()#проверяем уникальные значения

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [19]:
scoring['family_status'] = scoring['family_status'].str.lower()#приводим столбец к нижнему регистру
scoring['family_status'].unique()#проверяем изменения

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [20]:
scoring['gender'].unique()#проверяем уникальные значения

array(['F', 'M', 'XNA'], dtype=object)

In [21]:
#перезапишем таблицу без значения 'XNA' в "пол клиента"
scoring = scoring[scoring['gender'] != 'XNA']
scoring['gender'].unique()#проверяем изменения

array(['F', 'M'], dtype=object)

In [22]:
scoring['income_type'].unique()#проверяем уникальные значения

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [23]:
scoring['purpose'].unique()#проверяем уникальные значения

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Вывод

Отсутствующие значения составили 10% от исходных данных по значениям в столбцах: "общий трудовой стаж" и "ежемесячный доход". Возможно это тоже техническая ошибка при прочтении данных, так как анкета просто была бы не принята без заполнения всех значений по клиенту. В общем трудовом стаже отсутствующие значения были заменены на 'unknown', так как значения не будут участвовать в расчетах. В ежемесячном доходе большая разница между максимальными и минимальными значениями поэтому отсутствующие значения были заменены медианой. В столбце "дети" было одно отрицательное значение, скорей всего это ошибка выгрузки, было заменено на положительное. Так же в этом столбце есть 67 клиентов с двадцатью детьми, при этом нет промежуточных значений от 5 до 20. Думаю, здесь ошибка или при заполнении анкеты или при выгрузке данных и изначально эти значения были записаны как 2.0. Заменили все значения 20 на 2. В столбце "возраст клиента в годах" нулевые значения были заменены на средние арифметическое. В столбе "пол клиента" был обнаружен артефакт с значением 'XNA', перезаписали исходную таблицу без артефакта. Всю таблицу привели к нижнему регистру.

## **Замена типа данных**

In [24]:
#заменяем тип данных с вещественного на целое число
scoring['dob_years'] = scoring['dob_years'].astype('int')
scoring['total_income'] = scoring['total_income'].astype('int')
scoring.info()#проверим

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null object
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(6), object(6)
memory usage: 2.1+ MB


### Вывод

В столбцах "Общий трудовой стаж в днях" и "Ежемесячный доход" при помощи метода astype() заменили тип данных с вещественного на целое число для облегчения расчетов и для более легкого чтения значений.

### Обработка дубликатов

In [25]:
#посчитаем количество дубликатов
print('Дубликатов в таблице:', scoring.duplicated().sum())
#посмотрим на строки с дубликатами
scoring[scoring.duplicated() == True]

Дубликатов в таблице: 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,unknown,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,unknown,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,unknown,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,unknown,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,unknown,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,unknown,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,unknown,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,unknown,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,unknown,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


In [26]:
#удалим дубликаты
scoring = scoring.drop_duplicates().reset_index(drop = True)
print('Дубликатов в таблице:', scoring.duplicated().sum())#проверим

Дубликатов в таблице: 0


### Вывод

В таблице обнаружилось 71 дубликат. Для удаления дубликатов был применен метод .drop_duplicates() с изменением индексов и удалением старых из таблицы. Дубликаты могли появиться в результате сбоя в записи данных трудового стажа и ежемесячного дохода, так как во всех дубликатах эти значения отсутствовали, а ID у клиентов нет.

### Лемматизация

In [27]:
from pymystem3 import Mystem#импортируем библиотеку с функцией лемматизации на русском языке
m = Mystem()
#напишем функцию, которая будет идти по строкам столбца "цель получения кредита"  
#и каждый записанный текст в строке разделять на леммы
def purpose_list(row):
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    return lemmas
#создаем новый столбец с разделением целей кредита на леммы
scoring['purpose_lemmas'] = scoring.apply(purpose_list, axis = 1)
#посчитаем количество уникальных значений
scoring['purpose_lemmas'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

Выделила леммы в значениях столбца с целями получения кредита. Для этого написала функцию, которая идет по строкам столбца "цель получения кредита" и каждый записанный текст в строке разделяет на леммы.
Методом .apply()создала новый столбец с разделением целей кредита на леммы.
Методом .value_counts() посчитала какие цели встречаются чаще всего. 

### Категоризация данных

In [28]:
# разделим цели по категориям, напишем функцию, которая возвращает категорию цели получения кредита
def purpose_group(row):
    group = row['purpose_lemmas']
    if 'автомобиль' in group:
        return 'приобретение автомобиля'
    if 'свадьба' in group:
        return 'организация свадьбы'
    if 'недвижимость' in group or 'жилье' in group:
        return 'приобретение недвижимости'
    return 'получение образования'
#создадим новый столбец целей кредитов с разделением их по категориям
scoring['purpose_group'] = scoring.apply(purpose_group, axis = 1)
scoring['purpose_group'].value_counts()


приобретение недвижимости    10810
приобретение автомобиля       4306
получение образования         4013
организация свадьбы           2324
Name: purpose_group, dtype: int64

### Вывод

В основном целями в получения кредитов являются: приобретение недвижимости (сюда можно отнести цели такие как: ремонт недвижимости, строительство недвижимости и операции с недвижимостью), приобретение автомобиля, организация свадьбы и получение образования. Разделим их на категории для этого напишем функцию, которая в зависимости от условия возвращает нужную категорию цели получения кредита. Создадим новый столбец целей кредитов с разделением их по категориям и посчитаем какие категории целей встречаются чаще всего. В результате получили, что основной целью получения кредита является "приобретение недвижимости", эта цель составляет 50% от общего объема целей всех кредитов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
#сгруппируем таблицу по количеству детей и задолженности по возврату кредитов
#посчитаем количество строк по каждой категории и сумму задолженности по каждой категории
grouped_children = scoring.groupby('children').agg({'debt' : ['count', 'sum']})
#определим доли невозврата по числу детей
grouped_children['percentage_share'] = (grouped_children['debt']['sum'] / grouped_children['debt']['count']) * 100
#отсортируем таблицу по доле невозврата, по убыванию
grouped_children.sort_values(by = 'percentage_share', ascending = False)

debt       percentage_share
          count   sum                 
children                              
4            41     4         9.756098
2          2128   202         9.492481
1          4855   445         9.165808
3           330    27         8.181818
0         14090  1063         7.544358
5             9     0         0.000000

### Вывод

Зависимость между детьми и возвратом кредита есть: чем детей меньше, тем доля невозврата тоже меньше. 
Выборка, где у клиентов 3 и более детей, считаю не информативной, так как она очень маленькая.
Клиенты, у которых есть дети, несут более тяжелую финансовую нагрузку и у них возрастает вероятность непредвидимых трат, поэтому платежеспособность у клиентов, у которых нет детей, выше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
#сгруппируем таблицу по семейному положению и задолженности по возврату кредитов
#посчитаем количество строк по каждой категории и сумму задолженности по каждой категории
grouped_family_status = scoring.groupby('family_status').agg({'debt' : ['count', 'sum']})
#определим доли невозврата по семейному положению
grouped_family_status['percentage share'] = (grouped_family_status['debt']['sum'] / grouped_family_status['debt']['count']) * 100
#отсортируем таблицу по доле невозврата, по убыванию
grouped_family_status.sort_values(by = 'percentage share', ascending = False)

debt      percentage share
                       count  sum                 
family_status                                     
не женат / не замужем   2810  274         9.750890
гражданский брак        4150  388         9.349398
женат / замужем        12339  931         7.545182
в разводе               1195   85         7.112971
вдовец / вдова           959   63         6.569343

### Вывод

Зависимость между семейным положением и возвратом кредита есть.
Клиенты, которые состоят в браке, более надежные клиенты. Это обусловлено тем, что они часто выступают созаемщиками по кредиту, особенно по кредитам цели, которых приобретение недвижимости. Как правило владеют общим имуществом за счет, которого можно рассчитаться с банком. 
Если оба супруга работают, то могут поддержать друг друга в непредвидимых финансовых ситуациях, например, при потере работы, тем самым уменьшая риски невозврата кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
#найдем минимальное, максимальное значение и медиану по уровню дохода
print('Минимальное значение по доходу', scoring['total_income'].min(), '\nМаксимальное значение по доходу', scoring['total_income'].max())
median_total_income = scoring['total_income'].median()
print('Медиана', median_total_income)

Минимальное значение по доходу 20667 
Максимальное значение по доходу 2265604
Медиана 145017.0


In [32]:
#напишем функцию, которая на основании условий будет относить доход к определенной категории
#дополним функцию конструкцией try-except, которая при некорректном условии для кого-либо из 
#значений, будет выводить на экран сообщение: "Проверь условия".
def group(row):
    try:
        total_income = row['total_income']
        if total_income <= median_total_income * 0.31:
            return 'низкий'
        if median_total_income * 0.31 < total_income < median_total_income * 0.75:
            return 'ниже среднего'
        if total_income >= median_total_income * 1.25:
            return 'высокий'
        return 'средний'
    except:
        print('Проверь условия')
#создадим новый столбец с разделением по категориям доходов клиентов
scoring['income_group'] = scoring.apply(group, axis = 1)
    

In [33]:
#сгруппируем таблицу по доходу и задолженности по возврату кредитов
#посчитаем количество строк по каждой категории и сумму задолженности по каждой категории
scoring_grouped_income = scoring.groupby('income_group').agg({'debt' : ['count', 'sum']})
#определим доли невозврата по категориям
scoring_grouped_income['percentage share'] = (scoring_grouped_income['debt']['sum'] / scoring_grouped_income['debt']['count']) * 100
#отсортируем таблицу по доле невозврата, по убыванию
scoring_grouped_income.sort_values(by = 'percentage share', ascending = False)

debt      percentage share
              count  sum                 
income_group                             
средний        9615  830         8.632345
ниже среднего  5286  424         8.021188
высокий        6337  473         7.464100
низкий          215   14         6.511628

### Вывод

Доход клиентов был разделен на категории:
1. высокий, доход больше 1.25 медианы (больше 181271,25)
2. средний, доход от 0,75 до 1,25 медианы (от 108762,75 до 181271,25)
3. ниже среднего, доход от 0,31 до 0,75 медианы (от 44955,27 до 108762,75)
4. низкий, доход меньше 0,31 медианы (меньше 44955,27)
Самая маленькая доля невозврата у клиентов с высоким доходом.
Клиентов с низким уровнем дохода слишком мало для качественной оценки их платежеспособности.
Самая высокая доля невозврата у клиентов со средним уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [34]:
#сгруппируем категории целей кредитов по задолженности по возврату кредитов
#посчитаем количество строк по каждой категории и сумму задолженности по каждой категории
scoring_grouped = scoring.groupby('purpose_group').agg({'debt' : ['count', 'sum']})
#определим доли невозврата по категориям
scoring_grouped['percentage share'] = (scoring_grouped['debt']['sum'] / scoring_grouped['debt']['count']) * 100
#отсортируем таблицу по доле невозврата, по убыванию
scoring_grouped.sort_values(by = 'percentage share', ascending = False)

debt      percentage share
                           count  sum                 
purpose_group                                         
приобретение автомобиля     4306  403         9.359034
получение образования       4013  370         9.220035
организация свадьбы         2324  186         8.003442
приобретение недвижимости  10810  782         7.234043

### Вывод

Самая большая доля невозврата у клиентов, целью которых является покупка автомобиля и получение образования. Самая низкая доля невозврата по цели - приобретение недвижимости. Это объясняется тем, что клиенты более ответственно подходят к покупке недвижимости и банк в свою очередь более скрупулёзно оценивает клиента на предмет платежеспособности, а также обычно все кредиты, связанные с недвижимостью, имеют обеспечение и поручительство. 

### Шаг 4. Общий вывод

In [35]:
#посмотрим как наблюдения в нашей выборке распределены между целями кредита и уровнем дохода
#воспользуемся таблицей сопряженности
pd.crosstab(scoring['purpose_group'], scoring['family_status'])

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
purpose_group,,,,,
организация свадьбы,0,0,2324,0,0
получение образования,238,199,404,2595,577
приобретение автомобиля,281,218,434,2736,637
приобретение недвижимости,676,542,988,7008,1596


In [36]:
#посмотрим как наблюдения в нашей выборке распределены между целями кредита и количеством детей
#воспользуемся таблицей сопряженности
pd.crosstab(scoring['purpose_group'], scoring['children'])

children,0,1,2,3,4,5
purpose_group,,,,,,
организация свадьбы,1530,534,224,32,3,1
получение образования,2642,876,418,69,7,1
приобретение автомобиля,2845,974,415,60,10,2
приобретение недвижимости,7073,2471,1071,169,21,5


In [37]:
#посмотрим как наблюдения в нашей выборке распределены между целями кредита и уровнем дохода
#воспользуемся таблицей сопряженности
pd.crosstab(scoring['purpose_group'], scoring['income_group'])

income_group,высокий,ниже среднего,низкий,средний
purpose_group,,,,
организация свадьбы,680,591,22,1031
получение образования,1123,1027,45,1818
приобретение автомобиля,1282,1032,40,1952
приобретение недвижимости,3252,2636,108,4814


In [38]:
#посмотрим как наблюдения в нашей выборке распределены между уровнем дохода и семейным статусом
#воспользуемся таблицей сопряженности
pd.crosstab(scoring['income_group'], scoring['family_status'])

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
income_group,,,,,
высокий,371,203,1230,3705,828
ниже среднего,286,307,1024,2995,674
низкий,12,15,40,135,13
средний,526,434,1856,5504,1295


In [39]:
#посмотрим как наблюдения в нашей выборке распределены между уровнем дохода и количеством детей
#воспользуемся таблицей сопряженности
pd.crosstab(scoring['income_group'], scoring['children'])

children,0,1,2,3,4,5
income_group,,,,,,
высокий,4082,1484,643,114,10,4
ниже среднего,3576,1107,526,70,5,2
низкий,156,39,19,1,0,0
средний,6276,2225,940,145,26,3


На основании анализа данных можно сделать выводы: 

1.Количество детей и семейное положение влияют как на уровень дохода, так и на факт погашения кредита в срок. Уровень дохода и платёжеспособность клиента снижается при наличии и увеличении количества детей. Наличие детей подразумевает дополнительную финансовую нагрузку и повышает уровень непредвидимых расходов клиента. Клиенты, состоящие в браке, имеют более высокие доходы и более высокую платежеспособность. 

2.50% кредитной массы составляют кредиты с целью приобретения недвижимости (операции с недвижимостью, ремонт). Меньше берут и меньше выплачиваются кредиты с целями приобретения автомобиля и получения образования. 

В итоге кредиты больше берут клиенты, состоящие в браке, без детей и на приобретение недвижимости, они же и являются самыми платежеспособными клиентами.